In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchtext.vocab import GloVe
from utils.preprocess import MINDIterator
from utils.utils import getVocab,getLoss,getLabel,constructBasicDict,run_eval,run_train
from models.FIM import FIMModel

## setting up the *FIM* model

### define paths and hyperparameters, load data

all these hyper parameters are fixed according to the paper [\[29\] Fine-grained Interest Matching for Neural News Recommendation](https://www.aclweb.org/anthology/2020.acl-main.77.pdf)

- *mode*: data to read (*demo*/*small*/*large*)

- *batch_size*: size of each minibatch

- *title_size*: max word capacity of title

- *his_size*: max record capacity of click history

- *npratio*: number of negtive sampling

- *dilation_level*: levels of diferrent dilation rate

- *kernel_size*: size of 1dCNN kernel

- *filter_num*: number of kernels in 1D CNN, which is also embedding dimension of news/user

- *embedding_dim*: word embedding dimension

- *metrics*: metrics to be used in evaluating

- *gpu*: gpu ID if available

- *attrs*: the columns used to construct vocab

### Note 

Although I list *dilation_level* and *kernel_size*, some properties related to both variables are fixed in the FIM model because I don't have time to well-design my model to make it adapt to these variables dynamically. 

In [2]:
hparams = {
    'mode':'large',
    'batch_size':5,
    'title_size':18,
    'his_size':50,
    'kernel_size':3,
    'npratio':4,     
    'dilation_level':3,
    'filter_num':150,
    'embedding_dim':300,
    'metrics':'group_auc,ndcg@4,mean_mrr',
    'gpu':'cuda:0',
    'attrs': ['title','category','subcategory']
}

# customize your path here

news_file_train = 'D:/Data/NR_data/dev/news_train.tsv'
news_file_test = 'D:/Data/NR_data/dev/news_test.tsv'
behavior_file_train = 'D:/Data/NR_data/dev/behaviors_train.tsv'
behavior_file_test = 'D:/Data/NR_data/dev/behaviors_test.tsv'
save_path = 'models/model_param/FIM_'+ hparams['mode'] +'.model'

# if user2id,word2id,news2id haven't been constructed
if not os.path.exists('data/nid2idx_{}_{}.json'.format(hparams['mode'],'train')):
    constructBasicDict(news_file_train,behavior_file_train,hparams['mode'],'train',hparams['attrs'])

if not os.path.exists('data/nid2idx_{}_{}.json'.format(hparams['mode'],'test')):
    constructBasicDict(news_file_test,behavior_file_test,hparams['mode'],'test',hparams['attrs'])

device = torch.device(hparams['gpu']) if torch.cuda.is_available() else torch.device("cpu")

iterator_train = MINDIterator(hparams=hparams,mode='train',news_file=news_file_train,behaviors_file=behavior_file_train)

iterator_test = MINDIterator(hparams=hparams,mode='test',news_file=news_file_test,behaviors_file=behavior_file_test)

vocab_train = iterator_train.vocab
embedding = GloVe(dim=300,cache='.vector_cache')
vocab_train.load_vectors(embedding)

vocab_test = iterator_test.vocab
vocab_test.load_vectors(embedding)

In [3]:
# you can load my model or train yours
if os.path.exists(save_path):
    fimModel = FIMModel(vocab=vocab_train,hparams=hparams)
    fimModel.load_state_dict(torch.load(save_path))
    fimModel.to(device).eval()

else:
    fimModel = FIMModel(vocab=vocab_train,hparams=hparams).to(device)
    fimModel.train()

### train the model

In [ ]:
if fimModel.training:
    loss_func = getLoss(fimModel)
    optimizer = optim.Adam(fimModel.parameters(),lr=0.001)
    fimModel = run_train(fimModel,iterator_train,optimizer,loss_func, epochs=20, interval=15)

### test & evaluate

In [4]:
fimModel.eval()
fimModel.vocab = vocab_test
fimModel.npratio = -1
iterator_test.npratio = -1

run_eval(fimModel,iterator_test)

2191it [01:11, 30.70it/s]
[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] [0.6293467283248901, 0.5966985821723938, 0.6141831278800964, 0.6227760910987854, 0.6162933707237244, 0.6338029503822327, 0.6048542857170105, 0.6132485866546631, 0.6232218146324158, 0.6086212396621704, 0.6553585529327393, 0.5906630754470825, 0.6149079203605652, 0.651023805141449, 0.6219635605812073, 0.6128901839256287, 0.609609842300415, 0.5923672914505005, 0.6289182901382446, 0.6314775347709656, 0.6076305508613586, 0.6236132383346558]
[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0] [0.6372274160385132, 0.6035161018371582, 0.6110125184059143, 0.6016165018081665, 0.6345651149749756, 0.6117435693740845, 0.6288829445838928]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] [0.6101465821266174, 0.6493020057678223, 0.6389237642288208, 0.6306511163711548, 0.6297664046287537, 0.623110830783844, 0.598240911

{'group_auc': 0.5043, 'ndcg@4': 0.204, 'mean_mrr': 0.2214}

### save the model

In [ ]:
fimModel.npratio = 4
torch.save(fimModel.state_dict(), save_path)